In [1]:
import pandas as pd

file_name = "dataset.csv"

df = pd.read_csv(file_name)

# Auswahl der wichtigsten Features nach den höchsten absoluten Spearman Korrelationswerten
important_features = [
    "InvSampTime",
    "RadV_50per",
    "OfsDevTight",
    "InvSampTurn",
    "AmpDevStrict",
    "RadV_25per",
    "RawRadius",
    "AmpSyncCheck",
    "msg_timestamp",
    "OfsCos200",
    "OfsCos100",
    "ClCos300",
    "OfsSin200",
    "ClSin300",
    "time_to_fail",
    "status_75",
    "c_van17",
    "c_serial_number",
    "mileage",
    "ClSin150",
    "ClCos150",
    "OfsSin100",
    "EcuLifeT",
]

df = df[important_features]

C:\Users\Q548239\AppData\Local\Temp\ipykernel_9848\2850263671.py:5: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_name)


In [2]:
# Erstellen einer neuen Spalte 'Survival', die True ist, falls der 'status_75' Wert je Fahrzeug (c_serial_number) und je System (c_van17) nie "VALID" beträgt
# Sonst ist der Wert False
# Die Spalte gibt also an, ob die Komponente irgendwann im Datensatz ausfallen wird
df["Survival"] = df.groupby(["c_van17", "c_serial_number"])["status_75"].transform(
    lambda x: "VALID" in x.values
)
df["Survival"] = ~df["Survival"]

# berechnen der Spalte 'sick_time'
# ändern der Spalte msg_timestamp von absoluten zu relativen werten, damit Abhängigkeit größer wird, und Modell besser trainiert werden kann
min_timestamps = df.groupby(['c_van17', 'c_serial_number'])['msg_timestamp'].transform('min')
df['sick_time'] = df['msg_timestamp'] - min_timestamps


# entfernen der irrelevanten Spalten
df = df[df["Survival"] != True]
df = df.drop(columns=["c_van17", "c_serial_number", "status_75", "Survival"])

In [3]:
# Schreiben des Dataframes in eine neue csv Datei
df.to_csv("prep_dataset.csv", index=False)